In [ ]:
##Tomaudothi
!pip -q install ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output

def tinh_bac_dinh(ma_tran_ke):
    bac = []
    for i in range(len(ma_tran_ke)):
        bac.append(sum(ma_tran_ke[i]))
    return bac

def sap_xep_dinh_theo_bac(ds_dinh, bac):
    da_xep = []
    da_danh_dau = []
    for i in range(len(bac)):
        bac_max = -1
        vi_tri_max = -1
        for j in range(len(bac)):
            if j not in da_danh_dau and bac[j] > bac_max:
                bac_max = bac[j]
                vi_tri_max = j
        da_danh_dau.append(vi_tri_max)
        da_xep.append(ds_dinh[vi_tri_max])
    return da_xep

def tao_bang_chi_muc(ds_dinh):
    bang = {}
    for i in range(len(ds_dinh)):
        bang[ds_dinh[i]] = i
    return bang

def to_mau_do_thi(ma_tran_ke, ds_dinh, mau_khoi_tao):
    chi_muc = tao_bang_chi_muc(ds_dinh)
    bac = tinh_bac_dinh(ma_tran_ke)
    ds_dinh_sap_xep = sap_xep_dinh_theo_bac(ds_dinh, bac)

    tap_mau = {dinh: mau_khoi_tao.copy() for dinh in ds_dinh}
    ket_qua = {}

    for dinh in ds_dinh_sap_xep:
        danh_sach_mau = tap_mau[dinh]
        if len(danh_sach_mau) == 0:
            ket_qua[dinh] = None
            continue

        mau_duoc_chon = danh_sach_mau[0]
        ket_qua[dinh] = mau_duoc_chon

        hang_ke = ma_tran_ke[chi_muc[dinh]]
        for j in range(len(hang_ke)):
            if hang_ke[j] == 1 and mau_duoc_chon in tap_mau[ds_dinh[j]]:
                tap_mau[ds_dinh[j]].remove(mau_duoc_chon)

    return ket_qua

def parse_colors(s):
    return [c.strip() for c in s.split(",") if c.strip()]

title = widgets.HTML("<h3>Nhap mau va ma tran ke (0/1)</h3>")

n_input = widgets.BoundedIntText(
    value=9, min=1, max=30, step=1,
    description="So dinh:", style={"description_width":"80px"}
)

colors_input = widgets.Text(
    value="Blue, Red, Yellow, Green",
    description="Mau:",
    placeholder="VD: Blue, Red, Yellow, Green",
    style={"description_width":"80px"},
    layout=widgets.Layout(width="520px")
)

btn_build = widgets.Button(description="Tao ma tran", button_style="info")
btn_run = widgets.Button(description="Chay to mau", button_style="success")

adj_box = widgets.VBox([])
out = widgets.Output()

def build_adj_inputs(n):
    rows = []

    header = widgets.HBox(
        [widgets.Label(value="", layout=widgets.Layout(width="36px"))] + [
            widgets.Label(value=str(j+1), layout=widgets.Layout(width="44px")) for j in range(n)
        ]
    )
    rows.append(header)

    for i in range(n):
        row_cells = [widgets.Label(value=str(i+1), layout=widgets.Layout(width="36px"))]
        for j in range(n):
            cell = widgets.Dropdown(
                options=[0, 1],
                value=0,
                layout=widgets.Layout(width="44px")
            )
            row_cells.append(cell)
        rows.append(widgets.HBox(row_cells))

    adj_box.children = rows

def get_adj_matrix(n):
    A = [[0]*n for _ in range(n)]
    rows = adj_box.children[1:]
    for i in range(n):
        row = rows[i].children[1:]
        for j in range(n):
            A[i][j] = int(row[j].value)
    return A

def on_build(_):
    n = n_input.value
    build_adj_inputs(n)
    with out:
        clear_output()
        print("Da tao o nhap ma tran ke.")
        print("Dinh se tu dong la A, B, C, ...")

def on_run(_):
    with out:
        clear_output()
        n = n_input.value
        colors = parse_colors(colors_input.value)
        if len(colors) == 0:
            print("Danh sach mau rong.")
            return

        A = get_adj_matrix(n)
        ds_dinh = "".join([chr(ord('A') + i) for i in range(n)])

        kq = to_mau_do_thi(A, ds_dinh, colors)

        print("So dinh:", n)
        print("Danh sach dinh:", ds_dinh)
        print("Danh sach mau:", colors)
        print("Ket qua to mau:")
        for dinh in ds_dinh:
            print(f"Dinh {dinh} = {kq[dinh]}")

btn_build.on_click(on_build)
btn_run.on_click(on_run)

ui = widgets.VBox([
    title,
    widgets.HBox([n_input, btn_build, btn_run]),
    colors_input,
    widgets.HTML("<b>Nhap ma tran ke (0/1):</b>"),
    adj_box,
    out
])

display(ui)
on_build(None)


tinh_bac_dinh(ma_tran_ke)

Input: ma trận kề n x n

Output: danh sách bậc bac[i] = tổng hàng i (số đỉnh kề với đỉnh i)

Ý nghĩa: bậc đỉnh = số cạnh nối với đỉnh đó.

sap_xep_dinh_theo_bac(ds_dinh, bac)

Input:

ds_dinh: danh sách tên đỉnh (bạn dùng chuỗi “ABC...”)

bac: danh sách bậc theo thứ tự đỉnh

Output: danh sách đỉnh đã sắp theo bậc giảm dần.

Cách làm: chọn “bậc lớn nhất chưa chọn” từng bước (giống selection sort).

Mục đích: tô màu theo thứ tự đỉnh “quan trọng” trước (đỉnh có nhiều cạnh).

tao_bang_chi_muc(ds_dinh)

Tạo dictionary để tra nhanh:

ví dụ {'A':0, 'B':1, ...}

Mục đích: từ tên đỉnh → biết nó nằm ở hàng/cột nào trong ma trận kề.

to_mau_do_thi(ma_tran_ke, ds_dinh, mau_khoi_tao)

Bước 1: tính bậc và sắp xếp đỉnh theo bậc giảm dần

bac = tinh_bac_dinh(ma_tran_ke)

ds_dinh_sap_xep = sap_xep_dinh_theo_bac(ds_dinh, bac)

Bước 2: tạo “tập màu còn được phép” cho từng đỉnh

tap_mau = {dinh: mau_khoi_tao.copy() for dinh in ds_dinh}

Nghĩa là ban đầu đỉnh nào cũng có thể dùng tất cả màu.

Bước 3: duyệt từng đỉnh theo thứ tự đã sắp

Lấy danh sách màu còn lại: danh_sach_mau = tap_mau[dinh]

Chọn màu đầu tiên còn khả dụng: mau_duoc_chon = danh_sach_mau[0]

Gán kết quả: ket_qua[dinh] = mau_duoc_chon

Bước 4: “cấm màu” đó cho các đỉnh kề

Lấy hàng kề của đỉnh hiện tại: hang_ke = ma_tran_ke[chi_muc[dinh]]

Với mỗi j nếu hang_ke[j] == 1 thì đỉnh ds_dinh[j] kề với dinh

Nếu màu vừa chọn đang nằm trong tap_mau của đỉnh kề thì remove đi.

Trường hợp hết màu:

Nếu danh_sach_mau rỗng, bạn gán None (không tô được với bộ màu hiện có)


In [ ]:
##
!pip -q install ipywidgets
import copy
from heapq import heappush, heappop
import ipywidgets as widgets
from IPython.display import display, clear_output

rows = [1, 0, -1, 0]
cols = [0, -1, 0, 1]

class priorityQueue:
    def __init__(self):
        self.heap = []
    def push(self, key):
        heappush(self.heap, key)
    def pop(self):
        return heappop(self.heap)
    def empty(self):
        return not self.heap

class nodes:
    def __init__(self, parent, mats, empty_tile_posi, costs, levels):
        self.parent = parent
        self.mats = mats
        self.empty_tile_posi = empty_tile_posi
        self.costs = costs
        self.levels = levels
    def __lt__(self, nxt):
        if self.costs != nxt.costs:
            return self.costs < nxt.costs
        return self.levels < nxt.levels

def calculateCosts(mats, final, n):
    count = 0
    for i in range(n):
        for j in range(n):
            if mats[i][j] != 0 and mats[i][j] != final[i][j]:
                count += 1
    return count

def isSafe(x, y, n):
    return 0 <= x < n and 0 <= y < n

def newNodes(mats, empty_tile_posi, new_empty_tile_posi, levels, parent, final, n):
    new_mats = copy.deepcopy(mats)
    x1, y1 = empty_tile_posi
    x2, y2 = new_empty_tile_posi
    new_mats[x1][y1], new_mats[x2][y2] = new_mats[x2][y2], new_mats[x1][y1]
    costs = calculateCosts(new_mats, final, n)
    return nodes(parent, new_mats, new_empty_tile_posi, costs, levels)

def printMatrix(mats):
    n = len(mats)
    for i in range(n):
        print(" ".join(str(mats[i][j]) for j in range(n)))

def printPath(root):
    if root is None:
        return
    printPath(root.parent)
    printMatrix(root.mats)
    print()

def find_zero(mats):
    n = len(mats)
    for i in range(n):
        for j in range(n):
            if mats[i][j] == 0:
                return [i, j]
    return None

def mat_to_tuple(mats):
    return tuple(tuple(r) for r in mats)

def solve(initial, final, max_nodes=200000):
    n = len(initial)
    empty_tile_posi = find_zero(initial)
    if empty_tile_posi is None:
        print("Khong tim thay o trong (0) trong trang thai ban dau.")
        return

    pq = priorityQueue()
    costs = calculateCosts(initial, final, n)
    root = nodes(None, initial, empty_tile_posi, costs, 0)
    pq.push(root)

    visited = set()
    visited.add(mat_to_tuple(initial))

    expanded = 0

    while not pq.empty():
        minimum = pq.pop()
        expanded += 1

        if minimum.costs == 0:
            print("Da tim thay loi giai.")
            print("So buoc:", minimum.levels)
            print("So nut da mo rong:", expanded)
            print()
            printPath(minimum)
            return

        if expanded > max_nodes:
            print("Dung do vuot gioi han so nut mo rong.")
            return

        for i in range(4):
            new_tile_posi = [
                minimum.empty_tile_posi[0] + rows[i],
                minimum.empty_tile_posi[1] + cols[i],
            ]
            if isSafe(new_tile_posi[0], new_tile_posi[1], n):
                child = newNodes(
                    minimum.mats,
                    minimum.empty_tile_posi,
                    new_tile_posi,
                    minimum.levels + 1,
                    minimum,
                    final,
                    n,
                )
                key = mat_to_tuple(child.mats)
                if key not in visited:
                    visited.add(key)
                    pq.push(child)

    print("Khong tim thay loi giai.")

def build_grid(n, default=None):
    options = list(range(n*n))
    rows_ui = []
    for i in range(n):
        cells = []
        for j in range(n):
            v = 0
            if default is not None:
                v = default[i][j]
            dd = widgets.Dropdown(options=options, value=v, layout=widgets.Layout(width="72px"))
            cells.append(dd)
        rows_ui.append(widgets.HBox(cells))
    return widgets.VBox(rows_ui)

def read_grid(grid, n):
    mats = []
    for i in range(n):
        row = [int(cell.value) for cell in grid.children[i].children]
        mats.append(row)
    return mats

def is_valid_state(mat, n):
    flat = [x for row in mat for x in row]
    return len(flat) == n*n and set(flat) == set(range(n*n))

title = widgets.HTML("<h3>Giao dien nhap bai toan N-Puzzle</h3>")

n_input = widgets.BoundedIntText(
    value=3, min=2, max=6, step=1,
    description="Kich thuoc:", style={"description_width":"95px"}
)

limit_input = widgets.IntText(
    value=200000, description="Gioi han nut:", style={"description_width":"95px"},
    layout=widgets.Layout(width="240px")
)

btn_build = widgets.Button(description="Tao bang nhap", button_style="info")
btn_run = widgets.Button(description="Chay", button_style="success")

init_box = widgets.VBox([])
goal_box = widgets.VBox([])
out = widgets.Output()

def on_build(_):
    n = n_input.value
    init_default = [[0]*n for _ in range(n)]
    goal_default = [[0]*n for _ in range(n)]

    init_grid = build_grid(n, init_default)
    goal_grid = build_grid(n, goal_default)

    init_box.children = [widgets.HTML("<b>Trang thai ban dau</b>"), init_grid]
    goal_box.children = [widgets.HTML("<b>Trang thai dich</b>"), goal_grid]

    with out:
        clear_output()
        print("Da tao bang nhap.")
        print("Nhap day du cac so 0..", n*n-1)
        print("So 0 la o trong.")

def on_run(_):
    with out:
        clear_output()
        if len(init_box.children) < 2 or len(goal_box.children) < 2:
            print("Hay bam 'Tao bang nhap' truoc.")
            return

        n = n_input.value
        init_grid = init_box.children[1]
        goal_grid = goal_box.children[1]

        initial = read_grid(init_grid, n)
        final = read_grid(goal_grid, n)

        if not is_valid_state(initial, n):
            print("Trang thai ban dau khong hop le. Can dung du cac so 0..", n*n-1)
            return
        if not is_valid_state(final, n):
            print("Trang thai dich khong hop le. Can dung du cac so 0..", n*n-1)
            return

        solve(initial, final, max_nodes=int(limit_input.value))

btn_build.on_click(on_build)
btn_run.on_click(on_run)

ui = widgets.VBox([
    title,
    widgets.HBox([n_input, limit_input, btn_build, btn_run]),
    init_box,
    goal_box,
    out
])

display(ui)
on_build(None)



1) Phần thư viện và biến hướng đi

ipywidgets: tạo giao diện nhập trên Colab

heapq: dùng làm hàng đợi ưu tiên (priority queue)

copy.deepcopy: copy ma trận khi tạo trạng thái mới

rows/cols (hay moves): 4 hướng di chuyển của ô trống: xuống, trái, lên, phải.

2) Priority Queue và Node
priorityQueue

self.heap: danh sách dùng làm heap

push(key): đưa một node vào heap

pop(): lấy node “nhỏ nhất” ra (tùy theo __lt__ trong Node)

empty(): kiểm tra heap rỗng

nodes

Mỗi node là một trạng thái của bàn cờ, gồm:

parent: node cha (để truy vết đường đi)

mats: ma trận trạng thái hiện tại

empty_tile_posi: vị trí ô trống (số 0)

costs: heuristic (ở đây là số ô sai vị trí so với trạng thái đích, không tính 0)

levels: số bước đã đi từ đầu đến node này (độ sâu)

__lt__: quy định node nào “nhỏ hơn” để heap lấy ra trước.
Trong code UI mình đưa, mình ưu tiên costs nhỏ hơn, nếu bằng thì ưu tiên levels nhỏ hơn.

3) Heuristic và tạo node con
calculateCosts(mats, final, n)

Đếm số ô đang sai vị trí so với final

Bỏ qua ô 0

Đây là heuristic kiểu “Misplaced tiles”

newNodes(...)

Copy ma trận hiện tại

Hoán đổi ô trống với ô kề (tạo trạng thái mới)

Tính heuristic cho trạng thái mới

Trả về node con

isSafe(x, y, n)

Kiểm tra tọa độ có nằm trong bảng 0..n-1 không

4) In lời giải
printMatrix(mats)

In ma trận ra màn hình

printPath(root)

In đường đi bằng cách:

gọi đệ quy lên parent trước

rồi in trạng thái hiện tại
=> sẽ in theo đúng thứ tự từ đầu → đích.

5) Hàm giải solve(initial, final, max_nodes=...)

Các bước chính:

Tìm vị trí ô trống trong initial bằng find_zero

Tạo node gốc:

costs = calculateCosts(initial, final, n)

levels = 0

Đưa node gốc vào priority queue

Dùng visited để tránh lặp trạng thái đã gặp:

chuyển ma trận về tuple (mat_to_tuple) để hash được trong set

Vòng lặp:

Lấy node tốt nhất (cost thấp nhất) ra

Nếu minimum.costs == 0 nghĩa là trùng final → in lời giải và kết thúc

Nếu vượt max_nodes thì dừng (tránh chạy vô hạn)

Thử di chuyển ô trống 4 hướng:

nếu hợp lệ thì tạo node con

nếu node con chưa có trong visited thì push vào heap

Lưu ý: Thuật toán này là “best-first search theo heuristic” (ưu tiên trạng thái có ít ô sai). Nó giống A* ở chỗ dùng priority queue, nhưng A* chuẩn thường ưu tiên theo f = g + h (levels + costs). Ở code này bạn đang ưu tiên chủ yếu theo h (và phụ theo levels khi hòa).

6) Phần giao diện nhập (ipywidgets)
Các widget chính

n_input: nhập kích thước n (2..6)

limit_input: giới hạn số node mở rộng

btn_build: tạo bảng nhập theo n

btn_run: chạy giải

init_box: chứa bảng nhập trạng thái ban đầu

goal_box: chứa bảng nhập trạng thái đích

out: vùng in thông báo/kết quả

build_grid(n, default)

Tạo bảng n×n, mỗi ô là Dropdown cho phép chọn số từ 0..n*n-1.

read_grid(grid, n)

Đọc giá trị từ dropdown về thành ma trận Python.

is_valid_state(mat, n)

Kiểm tra trạng thái hợp lệ:

phải có đúng các số 0..n*n-1 (không thiếu, không trùng)


In [2]:
!pip -q install ipywidgets
import math, time, ipywidgets as w
from IPython.display import display, clear_output

X,O,EMPTY="X","O",None

def newb(n): return [[EMPTY]*n for _ in range(n)]
def acts(b):
    n=len(b)
    return [(i,j) for i in range(n) for j in range(n) if b[i][j] is None]
def lines(n):
    return [[(i,j) for j in range(n)] for i in range(n)] + [[(i,j) for i in range(n)] for j in range(n)] + \
           [[(i,i) for i in range(n)], [(i,n-1-i) for i in range(n)]]
def win(b):
    n=len(b)
    for ln in lines(n):
        v=b[ln[0][0]][ln[0][1]]
        if v and all(b[i][j]==v for i,j in ln): return v
    return None
def full(b): return all(c is not None for r in b for c in r)
def term(b): return win(b) is not None or full(b)

def h(b):
    n=len(b); s=0
    for ln in lines(n):
        xs=os=0
        for i,j in ln:
            xs += (b[i][j]==X)
            os += (b[i][j]==O)
        if xs and os:
            continue
        if xs: s += 10**xs
        if os: s -= 10**os
    return s

def scored_moves(b, player):
    res=[]
    for i,j in acts(b):
        b[i][j]=player
        res.append(((i,j), h(b)))
        b[i][j]=None
    res.sort(key=lambda x: x[1], reverse=(player==X))
    return [m for m,_ in res]

def mm_move_fast(b, d, topk, ms):
    deadline = time.time() + ms/1000.0
    memo={}

    def time_up():
        return time.time() >= deadline

    def key(st,who,dd):
        return (tuple(map(tuple,st)),who,dd)

    def mx(st,dd,a,bt):
        if time_up(): return h(st)
        k=key(st,"M",dd)
        if k in memo: return memo[k]
        wv=win(st)
        if wv==X: return 10**9
        if wv==O: return -10**9
        if full(st): return 0
        if dd==0: return h(st)
        v=-math.inf
        for i,j in scored_moves(st,X)[:topk]:
            st[i][j]=X
            v=max(v, mn(st,dd-1,a,bt))
            st[i][j]=None
            a=max(a,v)
            if bt<=a: break
        memo[k]=v
        return v

    def mn(st,dd,a,bt):
        if time_up(): return h(st)
        k=key(st,"N",dd)
        if k in memo: return memo[k]
        wv=win(st)
        if wv==X: return 10**9
        if wv==O: return -10**9
        if full(st): return 0
        if dd==0: return h(st)
        v=math.inf
        for i,j in scored_moves(st,O)[:topk]:
            st[i][j]=O
            v=min(v, mx(st,dd-1,a,bt))
            st[i][j]=None
            bt=min(bt,v)
            if bt<=a: break
        memo[k]=v
        return v

    best=-math.inf; mv=None
    for i,j in scored_moves(b,X)[:topk]:
        if time_up(): break
        b[i][j]=X
        sc=mn(b,d-1,-math.inf,math.inf)
        b[i][j]=None
        if sc>best:
            best=sc; mv=(i,j)
    if mv is None:
        am=acts(b)
        return am[0] if am else None
    return mv

def stt(b,turn):
    wv=win(b)
    if wv: return f"Ket thuc: {wv} thang"
    if full(b): return "Ket thuc: Hue"
    return f"Den luot {turn}"

n_in=w.BoundedIntText(value=3,min=3,max=10,description="n:")
d_in=w.BoundedIntText(value=3,min=1,max=6,description="depth:")
k_in=w.BoundedIntText(value=8,min=2,max=20,description="topk:")
t_in=w.BoundedIntText(value=200,min=50,max=2000,description="ms:")
btn=w.Button(description="Start",button_style="info")
grid=w.VBox(); out=w.Output()
S={"n":3,"b":newb(3),"btns":[],"turn":O}

def sync():
    n=S["n"]; b=S["b"]; over=term(b)
    for i in range(n):
        for j in range(n):
            bt=S["btns"][i][j]
            bt.description=" " if b[i][j] is None else b[i][j]
            bt.disabled=over or b[i][j] is not None or S["turn"]!=O

def click(i,j):
    def f(_):
        with out:
            clear_output()
            b=S["b"]
            if S["turn"]!=O or term(b):
                print(stt(b,S["turn"])); return
            b[i][j]=O
            S["turn"]=X
            sync()
            if term(b):
                print(stt(b,S["turn"])); return
            mv=mm_move_fast(b,int(d_in.value),int(k_in.value),int(t_in.value))
            if mv: b[mv[0]][mv[1]]=X
            S["turn"]=O
            sync()
            print(stt(b,S["turn"]))
    return f

def build(n):
    btns=[]; rows=[]
    for i in range(n):
        r=[]; hb=[]
        for j in range(n):
            bt=w.Button(description=" ",layout=w.Layout(width="44px",height="44px"))
            bt.on_click(click(i,j))
            r.append(bt); hb.append(bt)
        btns.append(r); rows.append(w.HBox(hb))
    return btns, w.VBox(rows)

def start(_):
    with out:
        clear_output()
        n=int(n_in.value)
        S["n"]=n; S["b"]=newb(n)
        S["btns"], ui = build(n)
        grid.children=[ui]

        mv=mm_move_fast(S["b"],int(d_in.value),int(k_in.value),int(t_in.value))
        if mv: S["b"][mv[0]][mv[1]]=X
        S["turn"]=O
        sync()
        print("May: X (di truoc), Nguoi: O")
        print(stt(S["b"],S["turn"]))

btn.on_click(start)
display(w.VBox([w.HBox([n_in,d_in,k_in,t_in,btn]), grid, out]))
start(None)


1) Biến và biểu diễn bàn cờ

X, O, EMPTY = "X", "O", None

newb(n): tạo bàn n×n toàn None.

Bàn cờ là list 2 chiều: b[i][j].

2) Sinh nước đi

acts(b): trả về danh sách ô trống (i, j).

3) Kiểm tra thắng / kết thúc

lines(n): tạo các “đường thắng”:

n hàng

n cột

2 đường chéo

win(b): duyệt từng đường, nếu tất cả ô giống nhau và khác None → trả về "X" hoặc "O".

full(b): kiểm tra hết ô trống chưa.

term(b): game kết thúc nếu có người thắng hoặc bàn đầy.

4) Heuristic (đánh giá nhanh)

h(b): chấm điểm trạng thái để dùng khi cắt depth / hết thời gian.

Với mỗi đường (hàng/cột/chéo):

đếm số X (xs) và số O (os)

nếu đường đó có cả X và O → bỏ qua (khó thắng trên đường đó)

nếu chỉ có X → cộng 10**xs (càng nhiều X càng tốt)

nếu chỉ có O → trừ 10**os (càng nhiều O càng nguy hiểm)

5) Sắp xếp nước đi để alpha-beta prune tốt hơn

scored_moves(b, player):

thử đặt tạm player vào từng ô trống

tính h(b) để “ước lượng nước đi tốt”

sắp xếp:

nếu player==X thì ưu tiên điểm cao trước

nếu player==O thì ưu tiên điểm thấp (vì O là MIN)

Mục đích: alpha-beta cắt tỉa nhiều hơn và chạy nhanh hơn.

6) Minimax + Alpha–Beta + giới hạn topk + giới hạn thời gian
mm_move_fast(b, d, topk, ms)

d: độ sâu minimax (bao nhiêu lớp)

topk: mỗi lượt chỉ xét top k nước đi tốt nhất (giảm nhánh)

ms: giới hạn thời gian cho 1 lần AI tính (tránh chạy quá lâu)

Bên trong:

deadline = now + ms

time_up(): hết thời gian thì trả True

memo: cache kết quả để khỏi tính lại (giảm thời gian)

mx(...) và mn(...)

mx: lượt X (MAX) → chọn giá trị lớn nhất

mn: lượt O (MIN) → chọn giá trị nhỏ nhất

Dừng trong 3 trường hợp:

hết thời gian → trả h(st)

trạng thái kết thúc → trả điểm cực lớn/nhỏ:

X thắng: 10**9

O thắng: -10**9

hòa: 0

đạt độ sâu dd==0 → trả h(st)

Alpha-beta:

MAX cập nhật alpha, nếu beta <= alpha thì break

MIN cập nhật beta, nếu beta <= alpha thì break

Chỉ duyệt scored_moves(... )[:topk] để giảm nhánh.

Chọn nước đi cuối cùng

Trong mm_move_fast: thử từng nước đi tốt nhất của X (topk)

Tính điểm bằng mn(...)

Chọn nước đi có điểm cao nhất

Nếu hết thời gian mà chưa chọn được → lấy đại 1 ô trống để khỏi lỗi